In [1]:
!pip install mysql-connector-python
!pip install PyMySQL

     |████████████████████████████████| 37.6 MB 1.2 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
import mysql.connector as MySQL 
import pymysql
import os

In [2]:
from sklearn.model_selection import cross_val_score,KFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier, AdaBoostClassifier, StackingClassifier
from sklearn import metrics
from sklearn import preprocessing
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import fbeta_score, f1_score,precision_score, recall_score,accuracy_score, roc_curve, auc
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split,RandomizedSearchCV
from numpy import float64, int64, number
from datetime import datetime
import json

C:\Users\HarshR-MBP_Windows\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [4]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

In [5]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [6]:
# host_name = "localhost"
# username = 'root'
# password = 'root' 
# db = "capstone"
# auth_plugin = 'mysql_native_password'
# connection = None

In [7]:
# # DB connection
# def connect_to_db(host_name, username, password, auth_plugin):
#     try:
#         connection = MySQL.connect(host=host_name, user = username, password = password, auth_plugin=auth_plugin)
#         # preparing a cursor object
#         cursorObject = connection.cursor()
#         query = "SELECT Serial_number,document_id,URL,Credibility_rating, ad_count,ad_max_size,css_definitions,page_rank,bitly_clicks,bitly_referrers,tweets,delicious_bookmarks,fb_clicks,fb_comments,fb_likes,fb_shares,fb_total,alexa_linksin,alexa_rank,commas,dots,exclamations,questions,spelling_errors,text_complexity,smog,category,JJ,NN,DT,VB,RB,num_ne,sum_ne,document_url_y,X1,X2,X3,X4,X5,X9,Total,Leik,Eijk,Tastle,Leik_3_4_6,correction,resp_HNC,Controversial,troia_label FROM WEB_RAW_DATA"
#         df_raw = pd.read_sql(query,con=connection)
#         # disconnecting from server
#         connection.close()
#     except MySQL.errorcode as err:
#         if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
#             print("already exists.")
#         else:
#             print(err.msg)
#         else:
#             print('Connected!')
#     return df_raw

In [8]:
# df_raw = connect_to_db(host_name, username, password, auth_plugin)

In [4]:
connection = pymysql.connect(
  host ="localhost",
  user ="root",
  passwd ="root",
  database = "capstone"
)
# preparing a cursor object
cursorObject = connection.cursor()
  
query = "SELECT Serial_number,document_id,URL,Credibility_rating, ad_count,ad_max_size,css_definitions,page_rank,bitly_clicks,bitly_referrers,tweets,delicious_bookmarks,fb_clicks,fb_comments,fb_likes,fb_shares,fb_total,alexa_linksin,alexa_rank,commas,dots,exclamations,questions,spelling_errors,text_complexity,smog,category,JJ,NN,DT,VB,RB,num_ne,sum_ne,document_url_y,X1,X2,X3,X4,X5,X9,Total,Leik,Eijk,Tastle,Leik_3_4_6,correction,resp_HNC,Controversial,troia_label FROM WEB_RAW_DATA"
cursorObject.execute(query)
columns = [col[0] for col in cursorObject.description]
records = cursorObject.fetchall()
   
# disconnecting from server
connection.close()

In [5]:
df_raw = pd.DataFrame(data=records,columns=columns)

In [6]:
df_raw.head(10)

,Serial_number,document_id,URL,Credibility_rating,ad_count,ad_max_size,css_definitions,page_rank,bitly_clicks,bitly_referrers,...,X9,Total,Leik,Eijk,Tastle,Leik_3_4_6,correction,resp_HNC,Controversial,troia_label
0,1,2199424.0,http://unemployment.ohio.gov/,3.714285714285714,0E-15,0E-15,106.000000000000000,6.000000000000000,9.000000000000000,5.000000000000000,...,0E-15,7.000000000000000,0.571428571000000,0.357142857000000,0.613196274000000,0.542857143000000,0E-15,2,U,3.000000000000000
1,2,2199427.0,http://www.law.cornell.edu/wex/unemployment_co...,4.428571428571429,4.000000000000000,75000.000000000000000,1824.000000000000000,5.000000000000000,12.000000000000000,1.000000000000000,...,0E-15,7.000000000000000,0.714285714000000,0.714285714000000,0.735143276000000,0.600000000000000,0E-15,3,U,5.000000000000000
2,3,2547110.0,http://kiem-tv.com/?q=node/4395,4.000000000000000,0E-15,0E-15,18.000000000000000,2.000000000000000,0E-15,0E-15,...,0E-15,7.000000000000000,0.714285714000000,0.571428571000000,0.762835715000000,0.657142857000000,0E-15,2,U,4.000000000000000
3,4,2547112.0,http://www.sciencedaily.com/releases/2012/09/1...,4.833333333333333,9.000000000000000,75000.000000000000000,383.000000000000000,3.000000000000000,45.000000000000000,9.000000000000000,...,0E-15,6.000000000000000,0.916666667000000,0.916666667000000,0.892660382000000,0.866666667000000,0E-15,3,U,5.000000000000000
4,5,2547113.0,http://articles.timesofindia.indiatimes.com/20...,3.888888888888889,2.000000000000000,75000.000000000000000,599.000000000000000,3.000000000000000,3.000000000000000,3.000000000000000,...,0E-15,9.000000000000000,0.722222222000000,0.611111111000000,0.756750140000000,0.688888889000000,0E-15,2,U,4.000000000000000
5,6,2201749.0,http://www.ofspirit.com/brucefife1.htm,3.875000000000000,0E-15,0E-15,0E-15,1.000000000000000,10.000000000000000,5.000000000000000,...,0E-15,8.000000000000000,0.812500000000000,0.750000000000000,0.822781986000000,0.800000000000000,0E-15,2,U,1.000000000000000
6,7,2201753.0,http://www.ashtarcommandcrew.net/forum/topics/...,2.750000000000000,16.000000000000000,96000.000000000000000,1850.000000000000000,0E-15,0E-15,0E-15,...,0E-15,8.000000000000000,0.375000000000000,-0.048611111000000,0.314413828000000,0.400000000000000,-1.000000000000000,C,U,1.000000000000000
7,8,2201756.0,http://www.mindbodygreen.com/0-4348/5-Ways-to-...,4.250000000000000,13.000000000000000,75000.000000000000000,524.000000000000000,1.000000000000000,335.000000000000000,38.000000000000000,...,0E-15,4.000000000000000,0.625000000000000,0.625000000000000,0.691800413000000,0.500000000000000,-1.000000000000000,2,U,1.000000000000000
8,9,2214309.0,http://www.co-optimus.com/review/1158/page/1/g...,4.428571428571429,4.000000000000000,75000.000000000000000,483.000000000000000,2.000000000000000,18.000000000000000,2.000000000000000,...,0E-15,7.000000000000000,0.714285714000000,0.714285714000000,0.735143276000000,0.600000000000000,0E-15,3,U,4.000000000000000
9,10,2569345.0,http://katieholmes.com/,3.400000000000000,9.000000000000000,75000.000000000000000,35.000000000000000,4.000000000000000,0E-15,0E-15,...,0E-15,5.000000000000000,0.800000000000000,0.800000000000000,0.815012042000000,0.840000000000000,0E-15,2,U,3.000000000000000


In [1]:
def regression_retrain():
  #removing the obvious column not going to used in the model
    cols_not_required = ['Serial_number','document_id','URL','Catrgory']
    # cols_not_required = ['document_id','URL']
    # make a copy of the raw data
    df_main = df_raw.copy()
    df_main.drop(cols_not_required,axis=1,inplace=True)
    
    #getting on the popular domain as in the list below
    array = ['com', 'org', 'gov', 'edu','net','uk','au','ca','in']
    df2 = df_main.loc[df_main['document_url_y'].isin(array)]
    
    # Dropping the missing values.
    df2=df2.fillna(0)
    # df2 = df2.drop(['category','ad_count', 'ad_max_size', 'css_definitions', 'page_rank',
    #    'bitly_clicks', 'bitly_referrers', 'tweets', 'delicious_bookmarks',
    #    'fb_clicks', 'fb_comments', 'fb_likes', 'fb_shares', 'fb_total',
    #    'alexa_linksin','num_ne', 'sum_ne','X1', 'X2', 'X3', 'X4', 'X5', 'X9', 'Total',
    #    'Leik', 'Eijk', 'Tastle','Leik 3 4 6', 'correction', 'resp_HNC', 'Controversial', 'troia_label'],axis=1)

    # Using label encoder to convert document_url_y into dummy
    le = preprocessing.LabelEncoder()
    df2.document_url_y = le.fit_transform(df2.document_url_y)
    df1=df2

    # defining x and y
    y = df1["Credibility_rating"]
    X = df1.drop("Credibility_rating",axis=1) 

    # Splitting the dataset
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)
    
    # performing preprocessing part
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)

    #reading previous metrics 

    path_of_file = 'regression_performance.json'
    # checking if size of file is 0
    if os.path.getsize(path_of_file) == 0:
        return "File is empty, train the inital model"
    else:
        #loading the json 
        with open('regression_performance.json', 'r') as f:
            previous_perf = json.load(f)
            if(len(previous_perf)!=0):
                previous_r2_score = previous_perf['output'][len(previous_perf['output'])-1]['test_r2_score']
            else :
                return "train the inital model Or Check the Json"
        f.close()

    # Random Forest model
    rf =RandomForestClassifier(n_estimators=250,random_state=1)
    rf_model = rf.fit(X_train,y_train)
    #y_train_pred = rf_model.predict(X_train)
    y_test_pred = rf_model.predict(X_test)

    # train r2_score = r2_score(y_train,y_train_pred)
    test_r2_score = r2_score(y_test,y_test_pred)

    perform_metrics ={'test_r2_score':test_r2_score}

    # compare old and new accuracy and then pickle the new model to use by the application, 
    # if retrained model gives better performance
    if test_r2_score>previous_r2_score:
        with open("./models/le_x_docURL.pkl",'wb') as file:
            pickle.dump(le_x_docURL,file)
        file.close()
        with open("./models/le_y.pkl",'wb') as file:
            pickle.dump(le_y,file)
        file.close()
        with open("./models/standardscaler.pkl",'wb') as file:
            pickle.dump(sc,file)
        file.close()
        with open("./models/category_model.pkl",'wb') as file:
            pickle.dump(rf_model,file)
        file.close()
        # storing the data in JSON format
        previous_perf['output'].append({"CapturedTime" : datetime.now().strftime("%d/%m/%Y %H:%M:%S"),
                                        "test_r2_score":test_r2_score}
                                       )
        # saving the upgraded accuracy along with other metrics and old values in Json
        with open("regression_performance.json", "w") as outfile:
            json.dump(previous_perf, outfile)
        outfile.close()

    return previous_perf['output'][len(previous_perf['output'])-1]



In [2]:
def classification_retrain():
    
    #removing the obvious column not going to used in the model
    cols_not_required = ['Serial_number','document_id','URL','Credibility_rating']
    
    # make a copy of the raw data
    df_main = df_raw.copy()
    df_main.drop(cols_not_required,axis=1,inplace=True)
    
    #getting on the popular domain as in the list below
    array = ['com', 'org', 'gov', 'edu','net','uk','au','ca','in']
    df2 = df_main.loc[df_main['document_url_y'].isin(array)]
    
    #filling all null category as 'unknown'
    df2["category"].fillna("Unknown", inplace = True)
    
    # Dropping the missing values.
    df2=df2.fillna(0)
    
    # Categorical features has to be converted into integer values for the model to process(one hot encoding).
    y_data = df2["category"]
    X_data = df2.drop(['category','ad_count', 'ad_max_size', 'css_definitions', 'page_rank',
       'bitly_clicks', 'bitly_referrers', 'tweets', 'delicious_bookmarks',
       'fb_clicks', 'fb_comments', 'fb_likes', 'fb_shares', 'fb_total',
       'alexa_linksin','num_ne', 'sum_ne','X1', 'X2', 'X3', 'X4', 'X5', 'X9', 'Total',
       'Leik', 'Eijk', 'Tastle','Leik_3_4_6', 'correction', 'resp_HNC', 'Controversial', 'troia_label'],axis=1)
    
    # select categorical features
    cat_x = X_data.select_dtypes(include=['object']).columns
    
    le_x_docURL = preprocessing.LabelEncoder()
    X_data.document_url_y = le_x_docURL.fit_transform(X_data.document_url_y)
    
    # label encode the target variable
    le_y = preprocessing.LabelEncoder()
    y = le_y.fit_transform(y_data)

    # Splitting the dataset
    X_train,X_test,y_train,y_test = train_test_split(X_data,y,test_size=0.3,random_state=0)
    
    # performing preprocessing part
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    #reading previous metrics 

    path_of_file = 'classification_performance.json'
    # checking if size of file is 0
    if os.path.getsize(path_of_file) == 0:
        return "File is empty, train the inital model"
    else:
        #loading the json 
        with open('classification_performance.json', 'r') as f:
            previous_perf = json.load(f)
            if(len(previous_perf)!=0):
                previous_accuracy=previous_perf['output'][len(previous_perf['output'])-1]['Accuracy']
            else :
                return "train the inital model Or Check the Json"
        f.close()
      
    # Random Forest model
    rf =RandomForestClassifier()
    rf_model = rf.fit(X_train,y_train)
    #y_train_pred = rf_model.predict(X_train)
    y_test_pred = rf_model.predict(X_test)

    precision = precision_score(y_test,y_test_pred, average='weighted')
    recall = recall_score(y_test,y_test_pred, average='weighted')
    accuracy = accuracy_score(y_test,y_test_pred)
    f1 = f1_score(y_test,y_test_pred, average='weighted')
    perform_metrics ={"Accuracy" : accuracy,"Recall" : recall,
            "Precision" : precision,"F1-score" : f1}
    
    # compare old and new accuracy and then pickle the new model to use by the application, 
    # if retrained model gives better performance
    if accuracy>previous_accuracy:
        with open("./models/le_x_docURL.pkl",'wb') as file:
            pickle.dump(le_x_docURL,file)
        file.close()
        with open("./models/le_y.pkl",'wb') as file:
            pickle.dump(le_y,file)
        file.close()
        with open("./models/standardscaler.pkl",'wb') as file:
            pickle.dump(sc,file)
        file.close()
        with open("./models/category_model.pkl",'wb') as file:
            pickle.dump(rf_model,file)
        file.close()
        # storing the data in JSON format
        previous_perf['output'].append({"CapturedTime" : datetime.now().strftime("%d/%m/%Y %H:%M:%S"),
                                        "Accuracy" : accuracy,
                                        "Recall" : recall,
                                        "Precision" : precision,
                                        "F1-score" : f1}
                                      )
        #saving the upgraded accuracy along with other metrics and old values in Json
        with open("classification_performance.json", "w") as outfile:
            json.dump(previous_perf, outfile)
        outfile.close()
     
    
    return previous_perf['output'][len(previous_perf['output'])-1]

In [3]:
classification_retrain()
#before running this, run the primary training of the model for the same json (classification_performance) for initial accuracy

NameError: ignored

In [ ]:
regression_retrain()